In [1]:
import pickle
import numpy as np
import torch

In [12]:
def get_matches_GT(root_path, frame1, frame2):
    first_frame = pickle.load(open(root_path + frame1, 'rb'))
    second_frame = pickle.load(open(root_path + frame2, 'rb'))
    # pc = frame['points'][:, :3]
    
    # pc_wraped, R = self.rotate_point_cloud(pc)

    first_frame_box_names = {}
    for f in first_frame['objects']:
        first_frame_box_names[f['name']] = f['box']

    second_frame_box_names = {}
    for f in second_frame['objects']:
        second_frame_box_names[f['name']] = f['box']
    # common_boxes = np.intersect1d(list(first_frame_box_names.keys()), list(second_frame_box_names.keys()))
    
    # boxes1 = []
    # boxes2 = []
    # for common_box in common_boxes:
    #     boxes1.append(first_frame_box_names[common_box])
    #     boxes2.append(second_frame_box_names[common_box])
    boxes1 = np.array(list(first_frame_box_names.values()))
    boxes2 = np.array(list(second_frame_box_names.values()))
    # skip this  pair if no keypoints
    if len(boxes1) <= 1 or len(boxes2) <= 1:
        # self.logger.info("Time elapsed for one item is (hh:mm:ss.ms) {}".format(time()-start_time))
        # return{
        #     'keypoints0': torch.zeros([0, 0, 2], dtype=torch.double),
        #     'keypoints1': torch.zeros([0, 0, 2], dtype=torch.double),
        #     'descriptors0': torch.zeros([0, 2], dtype=torch.double),
        #     'descriptors1': torch.zeros([0, 2], dtype=torch.double),
        #     'file_name': frame1
        # }
        return np.array([]), np.array([])
    
    matches_1_to_2 = np.nonzero(np.in1d(list(first_frame_box_names.keys()), list(second_frame_box_names.keys())))[0]
    matches_2_to_1 = np.nonzero(np.in1d(list(second_frame_box_names.keys()), list(first_frame_box_names.keys())))[0]
    all_matches = np.vstack((matches_1_to_2, matches_2_to_1))
    
    return matches_1_to_2, matches_2_to_1

In [15]:
matches_1_to_2, matches_2_to_1 = get_matches_GT("/home/jeevanandam/thesis/CenterPoint_new/data/test/annos/", "seq_0_frame_0.pkl", "seq_0_frame_1.pkl")

In [14]:
matches_1_to_2, matches_1_to_2

(array([], dtype=float64), array([], dtype=float64))

In [108]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
from scipy.sparse.csgraph import shortest_path
import torch
from torch_geometric.data import Data
from torch_geometric.utils import remove_self_loops
import torch.nn.functional as F
import cv2

from sklearn.neighbors import RadiusNeighborsTransformer, NearestNeighbors

exp_alpha = 0.2  # for the exp function
k = 7 # min number of neighbors
nbrs_knn_general = NearestNeighbors(n_neighbors=k, algorithm='ball_tree')
nbrs_knn= NearestNeighbors(n_neighbors=6, algorithm='ball_tree')

def get_neighbour_indices(positions, distance=0, knn=True,include_camera=False):
    
    if(knn == False):

        rnt = RadiusNeighborsTransformer(mode='distance', radius=distance)
        rnt_result = rnt.fit(positions)
        result = rnt_result.radius_neighbors(positions, return_distance=False)
        # if include_camera:
        #     result[0]=np.arange(result.shape[0])
        return result

    if(knn == True):
        nbrs_knn.fit(positions)
        result = nbrs_knn.kneighbors(positions, return_distance=False)

        return result
    
def get_edges_and_adjancey_matrix_from_indices(indices,self_edge=True):
    """Create 2xE and NxN edge and adjency matrix from list of lenght of number of neighbors for each node."""
    number_of_nodes = indices.shape[0]
    number_of_edges = 0

    for i in range(indices.shape[0]):
        number_of_edges += len(indices[i])

    edges = np.zeros([2, number_of_edges], dtype=int)
    adjancey_matrix = np.zeros([number_of_nodes, number_of_nodes], dtype=int)

    counter = 0
    for i in range(number_of_nodes):
        for j in range(len(indices[i])):
            # edges[:, counter] = [i, indices[i][j]]
            adjancey_matrix[i, indices[i][j]] = 1
            adjancey_matrix[ indices[i][j],i] = 1
            
    for i in range(number_of_nodes):
        for j in range(i+1 if self_edge else i ):
            if adjancey_matrix[i,j]==1:
                edges[:,counter]=[i,j]
                counter += 1

    edges=edges[:,:counter]
    return edges, adjancey_matrix
    
def create_graph(bb_pt_t, node_features, frame_id):
    positions=bb_pt_t
    # .detach().cpu().numpy()[:,3:6]

    distance=10
    # print(distance)
    indices = get_neighbour_indices(positions, distance, knn=False,include_camera=True)


    # positions_distance = np.stack((positions_distance_1, positions_distance_2,positions_distance_3), axis=-1)
    positions_distance=squareform(pdist(positions))
    # positions_distance[1:,0]=0
    edges, adjancey_matrix = get_edges_and_adjancey_matrix_from_indices(indices)
    edge_distances = np.array([positions_distance[edges[0, i], edges[1, i]] for i in range(0, edges.shape[1])])
    
    edge_attr = torch.tensor(np.expand_dims(edge_distances,1), dtype=torch.float32)
    edge_index = torch.tensor(edges, dtype=torch.int64)

    # x = bb_pt_t.float()
    
    # obj_list_vect=np.asarray([obj.get_kitti_vector() for obj in obj_list])
    # v=torch.tensor(obj_list_vect, dtype=torch.float32)

    # y = bb_gt_t.float()
    # print(feature_gt[:,-1])

    positions = torch.tensor(positions, dtype=torch.float32)
    

    #edge_index, edge_attr = remove_self_loops(edge_index, edge_attr)
    edge_attr = (distance-edge_attr)/ distance

    # return Data(x=x, y=y, edge_index=edge_index, edge_attr=edge_attr, positions=positions,frame_id=frame_id,pc=pc,pc_feat=pc_feat)
    return Data(x=node_features, edge_index=edge_index, edge_attr=edge_attr, positions=positions,frame_id=frame_id)

In [109]:
import os.path as osp
import os

import torch
from torch_geometric.data import Dataset, download_url, Data
import pickle
import numpy as np


class WaymoGraphDatasetSimple(Dataset):
    def __init__(self, root, transform=None, pre_transform=None, pre_filter=None):
        

        self.waymo_path = "/home/jeevanandam/thesis/CenterPoint_new/data/val_1/waymo_graph/raw/"
        self.features_path = "/home/jeevanandam/thesis/CenterPoint_new/data/val_1/waymo_graph/features/"
        # self.processed_dir = "/home/jeevanandam/thesis/CenterPoint_new/data/val_1/waymo_graph/"
        self.graph_file_names = []
        
        self.files = [self.waymo_path + f for f in os.listdir(self.waymo_path)]
        self.file_names = [f for f in os.listdir(self.waymo_path)]
        
        super().__init__(root, transform, pre_transform, pre_filter)
        
        # self.graph_file_names = [self.processed_dir + f + ".pt" for f in os.listdir(self.waymo_path)]

        print("init donee")
        

    @property
    def raw_file_names(self):
        return []

    @property
    def processed_file_names(self):
        return self.graph_file_names

    def download(self):
        # Download to `self.raw_dir`.
        pass

    def process(self):
        print("HEREEE")
        for frame_name in self.file_names:
            out_path = self.processed_dir + "/" + frame_name.split(".")[0] + ".pt"
            self.graph_file_names.append(out_path)
            # read point cloud,
            with open(self.waymo_path + frame_name, 'rb') as f:
                first_frame = pickle.load(f)
            with open(self.features_path+frame_name, 'rb') as f:
                node_features = pickle.load(f)['detections_gt']['centerpoint_features']

            first_frame_box_names = {}
            for f in first_frame['objects']:
                first_frame_box_names[f['name']] = f['box']

            boxes1 = np.array(list(first_frame_box_names.values()))
            nodes = boxes1[:, :3]
            # convert point cloud to Data object
            scene_graph = create_graph(nodes, node_features, frame_name)
            
            torch.save(scene_graph, out_path)
        return

    def __len__(self):
        return len(self.graph_file_names)

    def __getitem__(self, idx):
        data = torch.load(os.path.join(self.processed_dir,
                          self.graph_file_names[idx]))
        return data

    def len(self):
        return len(self.graph_file_names)

    def get(self, idx):
        data = torch.load(osp.join(self.processed_dir, f'data_{idx}.pt'))
        return data


In [110]:
dataset = WaymoGraphDatasetSimple(root="/home/jeevanandam/thesis/CenterPoint_new/data/val_1/waymo_graph/")

Processing...


HEREEE
init donee


Done!


In [111]:
g1 = torch.load("/home/jeevanandam/thesis/CenterPoint_new/data/val_1/waymo_graph/processed/seq_0_frame_0.pt")

In [139]:
g1

Data(x=[30, 64], edge_index=[2, 51], edge_attr=[51, 1], positions=[30, 3], frame_id='seq_0_frame_0.pkl')

In [136]:
g1_np = g1.edge_index.cpu().numpy()

In [137]:
g1_np

array([[ 0,  1,  2,  3,  4,  5,  6,  7,  8,  8,  9, 10, 11, 11, 12, 13,
        13, 14, 15, 15, 15, 16, 17, 18, 18, 18, 18, 19, 19, 20, 20, 20,
        21, 21, 22, 22, 23, 24, 24, 25, 26, 26, 26, 27, 28, 28, 29, 29,
        29, 29, 29],
       [ 0,  1,  2,  3,  4,  5,  6,  7,  5,  8,  9, 10,  4, 11, 12,  6,
        13, 14,  0, 12, 15, 16, 17,  0, 12, 15, 18,  0, 19, 12, 15, 20,
        16, 21, 17, 22, 23,  9, 24, 25, 16, 21, 26, 27,  1, 28,  0, 15,
        18, 19, 29]])

In [138]:
list(zip(g1_np[0],g1_np[1]))

[(0, 0),
 (1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 5),
 (8, 8),
 (9, 9),
 (10, 10),
 (11, 4),
 (11, 11),
 (12, 12),
 (13, 6),
 (13, 13),
 (14, 14),
 (15, 0),
 (15, 12),
 (15, 15),
 (16, 16),
 (17, 17),
 (18, 0),
 (18, 12),
 (18, 15),
 (18, 18),
 (19, 0),
 (19, 19),
 (20, 12),
 (20, 15),
 (20, 20),
 (21, 16),
 (21, 21),
 (22, 17),
 (22, 22),
 (23, 23),
 (24, 9),
 (24, 24),
 (25, 25),
 (26, 16),
 (26, 21),
 (26, 26),
 (27, 27),
 (28, 1),
 (28, 28),
 (29, 0),
 (29, 15),
 (29, 18),
 (29, 19),
 (29, 29)]